In [1]:
# Install Tensorflow Lite Model Maker
# &> /dev/null suppresses output
!pip install -q tflite-model-maker &> /dev/null

In [2]:
# Imports and check that we are using TF2.x
import numpy as np
import os

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [3]:
data_path = tf.keras.utils.get_file(
      'flower_photos',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      untar=True)

228818944/228813984 [==============================] - 1s 0us/step


In [4]:
data = DataLoader.from_folder(data_path)
train_data, test_data = data.split(0.9)

In [5]:
# Model maker abstract model design specifics like convolutional layers, dense layers, relu, loss, flatten etc
# Model maker does transfer learning - > uses existing pre-trained model as starting point and takes the things
# the model learned about how images are constructed then applying them to understand the flowers. 

# By default: tensorflow lite model maker uses "MobileNet" model, designed to recognize 1000 types of image
# Its logic of learning features to distinguish between 1000 classes are reused.
# Same features can be mapped to the 5 classes of flowers
model = image_classifier.create(train_data)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


103/103 [==============================] - 47s 147ms/step - loss: 0.8709 - accuracy: 0.7706
Epoch 2/5
103/103 [==============================] - 15s 142ms/step - loss: 0.6581 - accuracy: 0.8935
Epoch 3/5
103/103 [==============================] - 15s 142ms/step - loss: 0.6230 - accuracy: 0.9138
Epoch 4/5
103/103 [==============================] - 15s 149ms/step - loss: 0.6027 - accuracy: 0.9275
Epoch 5/5
103/103 [==============================] - 15s 149ms/step - loss: 0.5891 - accuracy: 0.9354


In [6]:
loss, accuracy = model.evaluate(test_data)

12/12 [==============================] - 6s 164ms/step - loss: 0.6411 - accuracy: 0.8965


In [7]:
model.export(export_dir='/mm_flowers')